In [10]:
import sys
sys.path.append("../")
import numpy as np
import matplotlib.pyplot as plt
import scripts.theodolite_utils as theodo_u
import importlib
import time
import pandas as pd
theodo_u = importlib.reload(theodo_u)
weather_data_path = "../data/weather_2022/Quebec/"

In [11]:
theodo_u = importlib.reload(theodo_u)
file = ["7016293_02-2022_P1H.csv","7016293_03-2022_P1H.csv","7016293_04-2022_P1H.csv","7016293_05-2022_P1H.csv",
        "7016293_06-2022_P1H.csv","7016293_07-2022_P1H.csv","7016293_11-2022_P1H.csv","7016293_12-2022_P1H.csv"]

data_sorted = []
for name_file in file:
    file_path = weather_data_path+name_file
    rows = pd.read_csv(file_path, chunksize=5000000)
    for i, chuck in enumerate(rows):
        chuck.to_csv(weather_data_path+'out{}.csv'.format(i))
    weather_condition = 'ND'
    for i in chuck.values:
        if str(i[9])!="nan" and str(i[13])!="nan" and str(i[23])!="nan":
            timestamp = time.mktime(time.strptime(str(i[4]), "%Y-%m-%d %H:%M")) + 5*3600  # Time zone in data are HNL (Haiwai)
            temperature = theodo_u.str_to_float_value(i[9])
            humidity = float(str(i[13]))
            pressure = theodo_u.str_to_float_value(i[23])*10
            if str(i[29]) != 'ND' :
                weather = str(i[29])
                weather_condition = str(i[29])
            else:
                weather = weather_condition
            data_sorted.append([timestamp,temperature,humidity,pressure,weather])
theodo_u.save_weather_data(data_sorted, weather_data_path+"data_sorted_2022.txt")

Conversion done !


In [12]:
theodo_u = importlib.reload(theodo_u)
Timestamp_to_find = 1647486500
data_weather = theodo_u.read_weather_data(weather_data_path+"data_sorted_2022.txt")
data_weather = np.array(data_weather)
time_weather = data_weather[:,0].astype(np.float64)
index, _ = theodo_u.findClosest(time_weather, Timestamp_to_find)
temperature, humidity, pressure, weather = theodo_u.interpolation_weather_data(Timestamp_to_find, data_weather, index)